# Premier League Statistics

The English Premier League is top level english football and it is contested between 20 clubs and operates on a system of promotion and relgation.
Every year millions of fans of the game come together to create imaginary teams  of real life footballers and participate in a game called Fantasy premier league. The fans score points based on their teams actual statistical performance or their perceived contribution on the field of play.
Being a participant of the game I know how important these stats are while creating a team. Hence I have used this data to create a [Tableau Dashboard](https://public.tableau.com/views/PremierLeagueStats_16112946940540/About?:language=en&:display_count=y&publish=yes&:origin=viz_share_link).

## Index

* [Downloading HTML files](#Downloading-HTML-files)
* [Scraping Data](#Scraping-Data)
* [Data Cleaning](#Data-Cleaning)
* [Extracting Data using API](#Extracting-Data-using-API)
* [Updating the CSV](#Updating-the-CSV-file-with-clean-data)
* [Checking for correlation](#Checking-Correlation)
* [Visualization](#Visualization)
* [Credits](#Credits)
***

## Downloading HTML files

The cell below will not be used right now. Data was extracted this way initially. I currently directly use beautiful soup library to extract data using an URL.

In [61]:
import os
import time
import requests
import sys

stat_list = ["stats","keepers","keepersadv","shooting","passing","passing_types","gca","defense","possession","playingtime","misc"]
def retirieve_pages():
    for stat in stat_list:
        link = "https://fbref.com/en/comps/9/{}/Premier-League-Stats".format(stat)
    
        texts = requests.get(link)
        text_utf = texts.text.encode('utf=8')
    
        if not os.path.exists("/Users/sunsu/Documents/Projects/Football/HTML/{}".format("statistics")):
            os.makedirs("/Users/sunsu/Documents/Projects/Football/HTML/{}".format("statistics"))
        with open("/Users/sunsu/Documents/Projects/Football/HTML/{}/{}.html".format("statistics",stat),"wb")as output:
            output.write(text_utf)
        sys.stdout.flush()
if __name__ == "__main__":
    start_time = time.time()
    retirieve_pages()
    stop_time = time.time()
    print("time taken {}".format(stop_time - start_time))


time taken 0.5855550765991211


## Scraping Data

In [62]:
import pandas as pd
import re
import sys, getopt
import csv
from bs4 import BeautifulSoup

def meta_data(stat):
    if stat == "stats":
        html = requests.get("https://fbref.com/en/comps/9/{}/Premier-League-Stats".format(stat),'rb')
        comm = re.compile("<!--|-->")
        soup = BeautifulSoup(comm.sub("",html.text),'lxml')
        all_table = soup.findAll("tbody")
        squad_stats= all_table[0]
        player_stats= all_table[1]
    
#finding player stats
        temp_player= {}
        stats_required = {"player","nationality","position","squad","games","games_starts","minutes","minutes_90s","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90"}
        rows = player_stats.find_all('tr')
        for row in rows:
            if (row.find("th",{"scope":"row"})!=None):
                for s in stats_required:
                    cell = row.find("td",{"data-stat": s})
                    if cell != None:
                        st = cell.text.strip().encode()
                        text= st.decode("utf-8")
                        if s in temp_player:
                            temp_player[s].append(text)
                        else:
                            temp_player[s] = [text]
        Player_Standard_Stats =  pd.DataFrame.from_dict(temp_player)
        Player_Standard_Stats.to_csv("Player_Standard_Stats.csv")
        
    if stat == "passing":
        html = requests.get("https://fbref.com/en/comps/9/{}/Premier-League-Stats".format(stat),'rb')
        comm = re.compile("<!--|-->")
        soup = BeautifulSoup(comm.sub("",html.text),'lxml')
        all_table = soup.findAll("tbody")
        squad_stats= all_table[0]
        player_stats= all_table[1]
    
#finding player stats
        temp_player= {}
        stats_required = {"player","passes_completed","passes","passes_pct","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","cards_yellow","passes_pct_long","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"}
        rows = player_stats.find_all('tr')
        for row in rows:
            if (row.find("th",{"scope":"row"})!=None):
                for s in stats_required:
                    cell = row.find("td",{"data-stat": s})
                    if cell != None:
                        st = cell.text.strip().encode()
                        text= st.decode("utf-8")
                        if s in temp_player:
                            temp_player[s].append(text)
                        else:
                            temp_player[s] = [text]
        Player_Passing_Stats =  pd.DataFrame.from_dict(temp_player)
        Player_Passing_Stats.to_csv("Player_Passing_Stats.csv")
        
        
    
    
if __name__ == "__main__":
    start_time = time.time()
    statistics = ["stats","passing"]
    for stats in statistics:
        meta_data(stats)
    stop_time = time.time()
    print("time taken {}".format(stop_time - start_time))

time taken 2.734525442123413


## Data Cleaning

In [63]:
player_stats = pd.read_csv("Player_Standard_Stats.csv")
passing = pd.read_csv("Player_Passing_Stats.csv")

In [64]:
stats = player_stats.merge(passing,left_on = "Unnamed: 0",right_on = "Unnamed: 0")
stats

,Unnamed: 0,goals_per90,assists_per90,games,position,goals_assists_per90,pens_made,assists,goals_assists_pens_per90,pens_att,...,passes_into_final_third,passes_completed,passes,passes_pct_long,crosses_into_penalty_area,assisted_shots,player_y,passes_into_penalty_area,passes_long,passes_short
0,0,0.00,0.10,11,DF,0.10,0,1,0.10,0,...,39,526,638,53.8,4,11,Patrick van Aanholt,20,80,305
1,1,0.56,0.09,18,FW,0.66,0,1,0.66,0,...,12,138,201,60.0,0,8,Tammy Abraham,2,15,108
2,2,0.23,0.23,19,FW,0.47,0,4,0.47,0,...,19,257,370,72.1,0,19,Che Adams,12,43,167
3,3,0.00,0.00,16,DF,0.00,0,0,0.00,0,...,36,627,742,70.5,0,3,Tosin Adarabioyo,1,217,131
4,4,0.00,0.00,2,GK,0.00,0,0,0.00,0,...,0,43,62,37.9,0,0,Adrián,0,29,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,476,0.00,0.00,2,"FW,DF",0.00,0,0,0.00,0,...,1,5,11,100.0,1,0,Andi Zeqiri,1,1,5
477,477,0.00,0.00,7,DF,0.00,0,0,0.00,0,...,37,496,552,69.2,0,1,Oleksandr Zinchenko,7,52,286
478,478,0.14,0.43,11,"FW,MF",0.57,0,3,0.57,0,...,33,352,470,46.2,4,17,Hakim Ziyech,19,93,195
479,479,0.27,0.00,15,DF,0.27,0,0,0.27,0,...,72,1005,1111,80.9,0,1,Kurt Zouma,2,251,245


In [65]:
stats[["Count","country"]] = stats["nationality"].str.split(" ",expand = True)
stats[["Position_1","Position_2"]] = stats["position"].str.split(",",expand = True)

In [66]:
col_name = ['games', 'games_starts',"Position_2","Position_1",'country',"squad","player_x"]
for col in col_name:
    first_col = stats.pop(col)
    stats.insert(0,col,first_col)
stats.duplicated("player_x")
names = stats["player_x"].value_counts()
names[names > 1]

Theo Walcott          2
Ruben Loftus-Cheek    2
Ross Barkley          2
Robert Snodgrass      2
Name: player_x, dtype: int64

In [67]:
l = ["Count","Unnamed: 0","nationality","position",'player_y']
for ls in l:
    stats = stats.drop([ls],axis =1)

## Extracting Data using API

In [68]:
url = 'https://fantasy.premierleague.com/api/element-summary/302/'
ur = requests.get(url)
ur

<Response [200]>

In [69]:
json = ur.json()
json.keys()

dict_keys(['fixtures', 'history', 'history_past'])

In [70]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [72]:
elements = pd.DataFrame(json.get("elements"))
teams = pd.DataFrame(json.get("teams"))

teams["name"] = teams["name"].replace(['Leicester','Leeds','Man City', 'Man Utd','Newcastle','Spurs',],['Leicester City','Leeds United','Manchester City','Manchester Utd','Newcastle Utd','Tottenham',])

In [73]:
exp = elements[['web_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points','status','goals_scored','clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps',"ict_index_rank", 'influence', 'creativity', 'threat','cost_change_event']]

In [74]:
exp['team'] = exp.team.map(teams.set_index('id').name)

<ipython-input-74-289e7b778b78>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp['team'] = exp.team.map(teams.set_index('id').name)


In [75]:
exp["player_team"] = exp[["web_name","team"]].apply(lambda x: ' '.join(x), axis=1)

<ipython-input-75-cf019c4ee528>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp["player_team"] = exp[["web_name","team"]].apply(lambda x: ' '.join(x), axis=1)


In [76]:
new = stats["player_x"].str.split(" ", n = 1, expand = True)

In [77]:
count = 0
stats["l_name"] = ""
for x in new[1]:
    stats["l_name"].iloc[count] = x
    count +=1

C:\Users\sunsu\Miniconda3\lib\site-packages\pandas\core\indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [78]:
stats["f_name"] = ""
count = 0
for x in new[0]:
        stats["f_name"].iloc[count] = x
        count +=1

In [79]:
stats["l_name"] = stats["l_name"].astype(str)
names = stats["f_name"].unique()

In [80]:
stats["player_squad"] = stats[["l_name","squad"]].apply(lambda x: ' '.join(x), axis=1)
stats["player2_squad"] = stats[["f_name","squad"]].apply(lambda x: ' '.join(x), axis=1)

In [81]:
statistic_1 = stats.merge(exp,left_on = "player_squad",right_on = "player_team",right_index=False)
statistic_2 = stats.merge(exp,left_on = "player2_squad",right_on = "player_team",right_index=False)
print(statistic_2[statistic_2["player_x"]=="James Justin"].index.values)
statistic_2 = statistic_2.drop([14,15])

[14]


In [82]:
statistic = pd.concat([statistic_1,statistic_2],ignore_index=True)
statistic = statistic.sort_index()

In [83]:
unwanted = ['team','web_name','l_name', 'player2_squad','f_name', 'player_squad',"player_team"]
for u in unwanted:
    statistic = statistic.drop([u],axis =1)
statistic = statistic.rename(columns={"player_x": "Player"})

## Updating the CSV file with clean data

In [84]:
statistic.to_csv("/Users/sunsu/Documents/Projects/Football/updated_data/Player_complete_Stats.csv")

In [85]:
statistic

,Player,squad,country,Position_1,Position_2,games_starts,games,goals_per90,assists_per90,goals_assists_per90,...,yellow_cards,red_cards,saves,bonus,bps,ict_index_rank,influence,creativity,threat,cost_change_event
0,Patrick van Aanholt,Crystal Palace,NED,DF,None,10,11,0.00,0.10,0.10,...,1,0,0,3,171,193,159.6,163.9,135.0,0
1,Tammy Abraham,Chelsea,ENG,FW,None,10,18,0.56,0.09,0.66,...,0,0,0,8,222,61,270.2,117.5,464.0,0
2,Che Adams,Southampton,ENG,FW,None,18,19,0.23,0.23,0.47,...,1,0,0,9,241,32,315.0,279.5,545.0,0
3,Tosin Adarabioyo,Fulham,ENG,DF,None,16,16,0.00,0.00,0.00,...,1,0,0,5,272,200,324.0,29.0,91.0,0
4,Sergio Agüero,Manchester City,ARG,FW,None,2,5,0.00,0.00,0.00,...,0,0,0,0,20,368,7.2,36.9,65.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,Rodrigo,Leeds United,ESP,MF,None,12,18,0.27,0.09,0.36,...,1,0,0,8,203,48,212.6,310.5,429.0,0
429,Cédric Soares,Arsenal,POR,DF,None,3,4,0.00,0.33,0.33,...,1,0,0,3,76,349,53.6,65.7,10.0,0
430,Trézéguet,Aston Villa,EGY,FW,None,8,11,0.00,0.12,0.12,...,0,0,0,0,78,136,115.8,122.2,343.0,0
431,Vitinha,Wolves,POR,MF,None,2,11,0.00,0.44,0.44,...,0,0,0,0,56,366,35.0,46.5,25.0,0


## Checking Correlation

In [86]:
passing_stats = ['passes_pct_long', 'passes_completed_medium', 'passes_into_final_third',
       'passes', 'passes_completed', 'passes_pct_medium', 'passes_short',
       'crosses_into_penalty_area', 'passes_into_penalty_area','passes_medium', 'passes_completed_long', 'passes_long',
       'passes_completed_short', 'progressive_passes', 'assisted_shots',
       'passes_pct_short', 'passes_pct']
for p in passing_stats:
    print("{} is correlated to assists by {}\n".format(p,(statistic[p].corr(statistic["assists"])*100)))

passes_pct_long is correlated to assists by -2.862895137564177

passes_completed_medium is correlated to assists by 18.305533981897824

passes_into_final_third is correlated to assists by 33.75076932827039

passes is correlated to assists by 33.971953106851096

passes_completed is correlated to assists by 30.11080480253269

passes_pct_medium is correlated to assists by -12.479301101485554

passes_short is correlated to assists by 48.30469939814475

crosses_into_penalty_area is correlated to assists by 44.308055765411574

passes_into_penalty_area is correlated to assists by 68.8713397041887

passes_medium is correlated to assists by 22.389041983586168

passes_completed_long is correlated to assists by 9.321102722667133

passes_long is correlated to assists by 11.460269683704723

passes_completed_short is correlated to assists by 45.41456631710945

progressive_passes is correlated to assists by 53.951548718237795

assisted_shots is correlated to assists by 78.60399580604066

passes_pct_s

Assisted shots that is key passes have a very high correlation with assists made. This can be really helpful for players while choosing their teams.

## Visualization

In [87]:
%%HTML
<div class='tableauPlaceholder' id='viz1611796766504' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Pr&#47;PremierLeagueStats_16112946940540&#47;Assists&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='PremierLeagueStats_16112946940540&#47;Assists' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Pr&#47;PremierLeagueStats_16112946940540&#47;Assists&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1611796766504');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1024px';vizElement.style.height='818px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1024px';vizElement.style.height='818px';} else { vizElement.style.width='100%';vizElement.style.height='3000px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

# Credits
Thank you [fbref](https://fbref.com/en/) for letting me scrape data